# Evaluating and Optimizing a RAG System


Installing Necessary libraries

In [ ]:
pip install Langchain sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.0 MB/s eta 0:00:00


In [ ]:
!pip install rouge


In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00


Importing Necessary libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge import Rouge
import numpy as np

# Web scraping function
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    return [p.get_text() for p in paragraphs]

# Clean scraped content
def clean_content(content_list):
    unwanted_items = {'Sign up', 'Sign in', 'Follow', '--', '15', 'Listen'}
    cleaned = [text for text in content_list if text and text not in unwanted_items]
    return cleaned

# Scrape and clean website content
url = 'https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7'
web_content = scrape_website(url)
cleaned_content = clean_content(web_content)

# Convert cleaned content to Documents
documents = [Document(page_content=text) for text in cleaned_content]

# Split the documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Set up a local language model
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    num_return_sequences=1,
    do_sample=True
)

# Create a HuggingFacePipeline language model
local_llm = HuggingFacePipeline(pipeline=pipe)

# Set up the retrieval-based QA system with a prompt template
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)

# RAG Pipeline function
def rag_pipeline(query):
    return qa.run(query)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<ipython-input-15-d1beb2f1a5ff>:63: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  local_llm = HuggingFacePipeline(pipeline=pipe)


 Creating an Example test Set

In [ ]:

# Example test set
test_questions = ["What is RAG?", "what embeddings are captured"]
test_answers = ["RAG is Retrieval-Augmented Generation", "Semantic embeddings are captured"]

# Get the RAG system answers
rag_answers = [rag_pipeline(question) for question in test_questions]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluating the Rag system

In [ ]:
from rouge import Rouge

# Initialize ROUGE scorer
rouge = Rouge()

# Example
scores = rouge.get_scores(rag_answers, test_answers, avg=True)

print("ROUGE Scores:", scores)


ROUGE Scores: {'rouge-1': {'r': 0.625, 'p': 0.02828535669586984, 'f': 0.05411602761310318}, 'rouge-2': {'r': 0.3333333333333333, 'p': 0.010101010101010102, 'f': 0.01960784285178778}, 'rouge-l': {'r': 0.625, 'p': 0.02828535669586984, 'f': 0.05411602761310318}}
